In [2]:
import sys
sys.path.insert(0, '..')
sys.path.insert(0, '../exposure_to_hazards/')

from config import DATA_DIR

In [3]:
from climada.entity import Exposures

In [4]:
import xarray as xr
from pathlib import Path


In [5]:
path_worldpop = DATA_DIR / "population/worldpop"


In [6]:
import numpy as np
dataset_list = [xr.open_dataset(path_worldpop / f"era5_compatible/all_{str(year)}_era5_compatible.nc") for year in np.arange(2000,2020)]
ds = xr.concat(dataset_list, dim='time')
ds = ds.where(ds != np.inf, 0)
ds = ds.where(ds<3.1e8, 0)

In [7]:
# extrapolate above 2020

In [9]:
import pandas as pd
import xarray as xr
import numpy as np

ds['time'] = xr.cftime_range(start='2000', periods=len(ds['time']), freq='AS')

# Exclude the value for the year 2020
ds = ds.sel(time=ds.time.dt.year != 2020)

# Extend the time range to include up to the year 2023
# Note: 'periods' should be the number of years from the start year to 2023
new_time = xr.cftime_range(start=ds.time.values[0], periods=(2023-2000)+1, freq='AS')

# Perform interpolation
ds_interpolated = ds.interp(time=new_time, method='linear', kwargs={"fill_value": "extrapolate"})
ds_interpolated = ds_interpolated.where(ds_interpolated >= 0, 0)
for time in np.arange(0,24):
    df = ds_interpolated.isel(time=time).to_dataframe().reset_index().drop(columns=['time'])
    exposures= Exposures()
    df = df.rename(columns={"pop":"value"})
    exposures.gdf = df
    exposures.write_hdf5(path_worldpop / f"climada_exposures/all_era5_025_compatible_{str(time+2000)}.hdf5")
ds_interpolated.to_netcdf(path_worldpop / 'era5_compatible/worldpop_all_2000_2023_era5_compatible.nc')


In [ ]:
##same thing for other age categories

In [11]:
import pandas as pd
import xarray as xr
import numpy as np

for sex in ["m","f"]:
    for age in ["0_1","65_70_75_80"]:
        dataset_list = [xr.open_dataset(path_worldpop / f"era5_compatible/{sex}_{age}_{str(year)}_era5_compatible.nc") for year in np.arange(2000,2020)]
        ds = xr.concat(dataset_list, dim='time')
        ds = ds.where(ds != np.inf, 0)
        ds = ds.where(ds<3.1e8, 0)
       
        # Ensure time coordinate is properly parsed as datetime objects
        ds['time'] = xr.cftime_range(start='2000', periods=len(ds['time']), freq='AS')

        # Exclude the value for the year 2020
        ds = ds.sel(time=ds.time.dt.year != 2020)

        # Extend the time range to include up to the year 2023
        # Note: 'periods' should be the number of years from the start year to 2023
        new_time = xr.cftime_range(start=ds.time.values[0], periods=(2023-2000)+1, freq='AS')

        # Perform interpolation
        ds_interpolated = ds.interp(time=new_time, method='linear', kwargs={"fill_value": "extrapolate"})
        ds_interpolated = ds_interpolated.where(ds_interpolated >= 0, 0)
        for time in np.arange(0,24):
            df = ds_interpolated.isel(time=time).to_dataframe().reset_index().drop(columns=['time'])
            exposures= Exposures()
            df = df.rename(columns={"pop":"value"})
            exposures.gdf = df
            exposures.write_hdf5(path_worldpop / f"climada_exposures/{sex}_{age}_era5_025_compatible_{str(time+2000)}.hdf5")

In [ ]:
# we merge both sexes for each age category

In [12]:
from pathlib import Path
import numpy as np
age = "0_1"
for year in np.arange(2003,2023):
    exp = Exposures.from_hdf5(path_worldpop / f"climada_exposures/f_{age}_era5_025_compatible_{year}.hdf5")
    exp2 = Exposures.from_hdf5(path_worldpop / f"climada_exposures/m_{age}_era5_025_compatible_{year}.hdf5")
    exp.gdf.value =exp2.gdf.value + exp.gdf.value
    exp.write_hdf5(path_worldpop / f"climada_exposures/{age}_era5_025_compatible_{year}.hdf5")
    
    

In [13]:
from pathlib import Path
import numpy as np
age = "65_70_75_80"
for year in np.arange(2003,2023):
    exp = Exposures.from_hdf5(path_worldpop / f"climada_exposures/f_{age}_era5_025_compatible_{year}.hdf5")
    exp2 = Exposures.from_hdf5(path_worldpop / f"climada_exposures/m_{age}_era5_025_compatible_{year}.hdf5")
    exp.gdf.value =exp2.gdf.value + exp.gdf.value
    exp.write_hdf5(path_worldpop / f"climada_exposures/{age}_era5_025_compatible_{year}.hdf5")
    
    